In [1]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Defina o caminho para o seu arquivo .parquet
caminho_do_arquivo = '/content/drive/MyDrive/02 - Economia (UFPE)/2025.1/Tópicos de Macro (Economia do clima)/analise-saneamento-chuva-extrema-pe/build/input/preciptation-zenodo/pr_3.2.3.parquet'

print(f"Lendo o arquivo: {caminho_do_arquivo}")

Lendo o arquivo: /content/drive/MyDrive/02 - Economia (UFPE)/2025.1/Tópicos de Macro (Economia do clima)/analise-saneamento-chuva-extrema-pe/build/input/preciptation-zenodo/pr_3.2.3.parquet


In [2]:
###CODIGO DOS MUNICIPIOS BRASILEIROS###

url = "https://servicodados.ibge.gov.br/api/v1/localidades/estados/11|12|13|14|15|16|17|21|22|23|24|25|26|27|28|29|31|32|33|35|41|42|43|50|51|52|53/municipios"

response = requests.get(url)

municipios = response.json()
# Criar DataFrame com os códigos e nomes dos municípios
dados = []
for municipio in municipios:
    try:
        dados.append({
            "id": municipio["id"],
            "nome": municipio["nome"],
            "uf": municipio["microrregiao"]["mesorregiao"]["UF"]["id"],
            "nome-uf": municipio["microrregiao"]["mesorregiao"]["UF"]["nome"],
            "regiao": municipio["microrregiao"]["mesorregiao"]["UF"]["regiao"]["nome"]
        })
    except (TypeError, KeyError):
        # pula o município com dados incompletos
        continue

cod_municipios = pd.DataFrame(dados)

codigos_pe = cod_municipios['id'][cod_municipios['nome-uf'] == 'Pernambuco'].to_list()

# Carregue apenas os dados de PE para economizar memória
df_pr_pe = pd.read_parquet(
    caminho_do_arquivo,
    filters=[('code_muni', 'in', codigos_pe)]
)

print("Dados de precipitação carregados. Renomeando colunas...")

# Renomear colunas para facilitar o uso
df_pr_pe = df_pr_pe.rename(columns={
    'code_muni': 'cod_municipio',
    'date': 'data',
    'value': 'precipitacao_media'
})

df_pr_pe = df_pr_pe.drop(columns=['name'])

# Converter a coluna de data para o formato datetime do pandas
df_pr_pe['data'] = pd.to_datetime(df_pr_pe['data'])

# Extrair o ano para facilitar a agregação
df_pr_pe['ano'] = df_pr_pe['data'].dt.year

Dados de precipitação carregados. Renomeando colunas...


In [3]:
# ---
# Passo 2: Calcular a variável 'Dias_Chuva_Extrema'
# ---
print("Calculando o limiar de chuva extrema para cada município...")

# Calcular o percentil 95 da chuva diária PARA CADA MUNICÍPIO
limiares = df_pr_pe.groupby('cod_municipio')['precipitacao_media'].quantile(0.95)

# Transformar a série em um DataFrame para o merge
limiares_df = limiares.reset_index()
limiares_df.rename(columns={'precipitacao_media': 'limiar_p95'}, inplace=True)

# Juntar os limiares de volta ao DataFrame principal
df_pr_pe = pd.merge(df_pr_pe, limiares_df, on='cod_municipio', how='left')

# Criar uma coluna 'extremo' (1 se a chuva excedeu o limiar, 0 caso contrário)
df_pr_pe['extremo'] = (df_pr_pe['precipitacao_media'] > df_pr_pe['limiar_p95']).astype(int)

print("Contando os dias de chuva extrema por ano para cada município...")

# Agrupar por município e ano e somar os dias extremos
dias_chuva_extrema = df_pr_pe.groupby(['cod_municipio', 'ano'])['extremo'].sum()

# Converter para DataFrame
df_chuva_extrema_final = dias_chuva_extrema.reset_index()
df_chuva_extrema_final.rename(columns={'extremo': 'dias_chuva_extrema'}, inplace=True)

print("\n--- Variável 'dias_chuva_extrema' criada com sucesso! ---")

print("Calculando o volume total de chuva nos dias extremos...")

# Crie uma nova coluna 'volume_extremo'.
# Ela será igual à precipitação se o dia for extremo (extremo=1), e zero caso contrário.
df_pr_pe['volume_extremo'] = np.where(
    df_pr_pe['extremo'] == 1,           # Condição: se o dia foi extremo
    df_pr_pe['precipitacao_media'], # Valor se verdadeiro: a própria chuva do dia
    0                                # Valor se falso: zero
)

# Agora, agrupe por município e ano e SOME o volume nos dias extremos
volume_chuva_extrema = df_pr_pe.groupby(['cod_municipio', 'ano'])['volume_extremo'].sum()

# Converter a série final para um DataFrame
df_volume_extremo_final = volume_chuva_extrema.reset_index()
df_volume_extremo_final.rename(columns={'volume_extremo': 'volume_total_chuva_extrema_mm'}, inplace=True)

print("\n--- Variável 'volume_total_chuva_extrema_mm' criada com sucesso! ---")

df_chuva_extrema_final = pd.merge(df_chuva_extrema_final, df_volume_extremo_final, on=['cod_municipio', 'ano'])
df_chuva_extrema_final = df_chuva_extrema_final[df_chuva_extrema_final['ano'].between(2002, 2021)].copy()

Calculando o limiar de chuva extrema para cada município...
Contando os dias de chuva extrema por ano para cada município...

--- Variável 'dias_chuva_extrema' criada com sucesso! ---
Calculando o volume total de chuva nos dias extremos...

--- Variável 'volume_total_chuva_extrema_mm' criada com sucesso! ---


In [4]:
df_chuva_extrema_final.to_parquet('/content/drive/MyDrive/02 - Economia (UFPE)/2025.1/Tópicos de Macro (Economia do clima)/analise-saneamento-chuva-extrema-pe/build/output/precipitacao_pe_2002_2021.parquet', index=False)